## 기본 세팅 코드

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from time import time
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Dropout, Input, Lambda
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

/Users/mark-lee/venv/watchout/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%matplotlib inline
sys.path.append(os.path.join(os.getcwd(),"models"))

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

MODEL_NAME = 'faster_rcnn_resnet101_coco_11_06_2017'
PATH_TO_CKPT = os.path.join(os.getcwd(),'watchout/models/fasterRCNN/graph/frozen_inference_graph.pb')
PATH_TO_LABELS = os.path.join(os.getcwd(),'watchout/data/deepfashion_label_map.pbtxt')
NUM_CLASSES = 50
detection_graph = tf.Graph()

with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## 학습 데이터 세팅

In [3]:
# Size, in inches, of the output images.
IMAGE_SIZE = (15, 10)

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

## helper

In [71]:
import pandas as pd
import random
import warnings
warnings.filterwarnings('ignore')

def create_path(subpath,target):
    return subpath+"/"+target

def create_image_path(path):
    return os.path.join(os.getcwd(),'watchout/data/raw_deepfashion_dataset/Img/'+path)

def random_sample(batch=1):
    ann = Anno(os.path.join(os.getcwd(), 'watchout/data/raw_deepfashion_dataset/Anno'))
    t_labels = []
    t_paths = []
    p_paths = []
    n_labels = []
    n_paths = []
    for i in range(batch):
        
        ran = random.randrange(0,len(ann))
        target_path, target_label = (ann.loc[ran]['image_name'],ann.loc[ran]['category_label'])
        positive_ran = random.choice(ann[ann['category_label']==target_label].index.values)
        positive_path = ann[ann['category_label']==target_label].loc[positive_ran]['image_name']

        while target_path == positive_path:
            print('loop with "target_path == positive_path"')
            positive_ran = random.choice(ann[ann['category_label']==target_label].index.values)
            positive_path = ann[ann['category_label']==target_label].loc[positive_ran]['image_name']    

        negative_ran = random.choice(ann[ann['category_label']!=target_label].index.values)
        negative_path = ann[ann['category_label']!=target_label].loc[negative_ran]['image_name']
        negative_label = ann[ann['category_label']!=target_label].loc[negative_ran]['category_label']
        
        t_labels.append(target_label)
        t_paths.append(create_image_path(target_path))
        p_paths.append(create_image_path(positive_path))
        n_labels.append(negative_label)
        n_paths.append(create_image_path(negative_path))
        
    return (t_labels, t_paths, p_paths), (n_labels,n_paths)
    
def Anno(path,head=False):
    category_path =create_path(path,"list_category_img.txt")
    category_data = pd.read_csv(category_path,sep=r"\s*",skiprows=[0],header=0)
    if head:
        return category_data.head(100)
    return category_data

def promising_box_index(scores):
    for idx, score in enumerate(scores):
        if score < 0.8: # 0.8 이상의 box만 리턴!
            return idx
    return len(scores)

def promising_boxes(boxes,scores):
    promising_idx = promising_box_index(scores)
    return tf.slice(tf.squeeze(boxes),[0,0],[promising_idx,4])

def images_from_paths(targets, positives, negatives):
    assert len(targets) == len(positives) and len(positives) == len(negatives)
    t_list = []
    p_list = []
    n_list = []
    for t,p,n in zip(targets,positives,negatives):
        t_single, p_single, n_single = image_from_path(t,p,n)
        t_list.append(t_single)
        p_list.append(p_single)
        n_list.append(n_single)
    
    return np.array(t_list), np.array(p_list), np.array(n_list)

def image_from_path(target, positive, negative):
    tmp_a = Image.open(target)
    tmp_p = Image.open(positive)
    tmp_n = Image.open(negative)
    
    # arr = array(img, dtype="float32")
    
    anchor = tmp_a.copy()
    anchor = anchor.resize((299, 299), Image.ANTIALIAS)
    anchor = np.array(anchor, dtype="float32")
    pos = tmp_p.copy()
    pos = pos.resize((299,299), Image.ANTIALIAS)
    pos = np.array(pos, dtype="float32")
    neg = tmp_n.copy()
    neg = neg.resize((299,299), Image.ANTIALIAS)
    neg = np.array(neg, dtype="float32")
    
    tmp_a.close()
    tmp_p.close()
    tmp_n.close()
    
    return anchor/255.0, pos/255.0, neg/255.0

def euclidean_distance(vects):
    anchor, target = vects
    #return K.sqrt(K.maximum(K.sum(K.square(anchor - target), axis=1, keepdims=True), K.epsilon()))
    return K.sum(K.square(anchor - target), axis=1, keepdims=True)

def triplet_loss(y_true, y_pred):
        margin = K.constant(0.2)
        return K.mean(K.maximum(K.constant(0), K.square(y_pred[:,0,0]) - K.square(y_pred[:,1,0]) + margin))

def get_triplet_model(intermediate=False):
    inceptionv3_input = Input(shape=(299,299,3))
    inceptionv3_f = InceptionV3(include_top=False, weights='imagenet', input_tensor=inceptionv3_input)
    net = inceptionv3_f.output
    net = Flatten(name='flatten')(net)
    net = Dense(1024, activation='relu', name='embedded')(net)
    
    base_model = Model(inceptionv3_f.input, net, name='base_inceptionv3')
    if intermediate:
        return base_model
    
    input_anchor = Input(shape=(299,299,3),name='input_anchor')
    input_positive = Input(shape=(299,299,3),name='input_pos')
    input_negative = Input(shape=(299,299,3),name='input_neg')
    
    net_anchor = base_model(input_anchor)
    net_positive = base_model(input_positive)
    net_negative = base_model(input_negative)
    if intermediate:
        return Model([input_anchor,input_positive,input_negative],
                     [net_anchor,net_positive,net_negative], name='triplet_model')
    
    d_positive = Lambda(euclidean_distance, name='d_pos')([net_anchor, net_positive])#euclidean_distance((net_anchor, net_positive))
    d_negative = Lambda(euclidean_distance, name='d_neg')([net_anchor, net_negative])#euclidean_distance((net_anchor, net_negative))
    d_stacked = Lambda(lambda vects: K.stack(vects, axis=1), name='d_stacked')([d_positive, d_negative])
    
    triplet_model = Model([input_anchor,input_positive,input_negative], d_stacked, name='triplet_model')
    return triplet_model

def get_detector_graph():
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return detection_graph

def get_latest_weights_and_global_step(_path):
    files = os.listdir(path=_path)
    print('weights list:'+str(files))
    files.reverse()
    return files[0],int(files[0].split('-')[1].split('.')[0])

def get_train_data(_batch,d_sess):
    (t_lbl, t_path, p_path), (n_lbl, n_path) = random_sample(_batch)

    anchor, positive, negative = images_from_paths(t_path, p_path, n_path)

    print('초기 anchor,positive,negative shape')
    print(anchor.shape)
    print(positive.shape)
    print(negative.shape)

    for idx,single_batch in enumerate(t_path):

        image = Image.open(single_batch)
        image_np = load_image_into_numpy_array(image)
        image_np_expanded = np.expand_dims(image_np, axis=0)
        (_image_tensor,_boxes, scores, classes, num) = d_sess.run(
          [image_tensor, detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})

        target_boxes = promising_boxes(_boxes, scores[0])
        len_target_boxes = int(target_boxes.shape[0])

        cropped_images = []
        for box in target_boxes.eval().tolist():
            current_cropped_image = tf.image.crop_and_resize(image=image_np_expanded/255.0,
                                                             boxes=[box],
                                                             box_ind=[0],
                                                             crop_size=[299,299])
            cropped_images.append(current_cropped_image[0])
            anchor= np.append(anchor,np.expand_dims(current_cropped_image[0].eval(),axis=0),axis=0)
            positive = np.append(positive,np.expand_dims(positive[idx],axis=0),axis=0)
            negative = np.append(negative,np.expand_dims(negative[idx],axis=0),axis=0)
    
    print('후기 anchor, positive, negative shape')
    print(anchor.shape)
    print(positive.shape)
    print(negative.shape)

    assert anchor.shape[0] == positive.shape[0] and positive.shape[0] == negative.shape[0]
    
    return anchor, positive, negative

## OD + Triplet 결합 & Triplet만 학습하는 코드

In [91]:
import warnings
warnings.filterwarnings('ignore')

triplet_graph = tf.Graph()

"""with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')"""
a = None
b = None
c = None

def get_triplet_model(intermediate=False):
    inceptionv3_input = Input(shape=(299,299,3))
    inceptionv3_f = InceptionV3(include_top=False, weights='imagenet', input_tensor=inceptionv3_input)
    net = inceptionv3_f.output
    net = Flatten(name='flatten')(net)
    net = Dense(1024, activation='relu', name='embedded')(net)
    
    base_model = Model(inceptionv3_f.input, net, name='base_inceptionv3')
    if intermediate:
        return base_model
    
    input_anchor = Input(shape=(299,299,3),name='input_anchor')
    input_positive = Input(shape=(299,299,3),name='input_pos')
    input_negative = Input(shape=(299,299,3),name='input_neg')
    
    net_anchor = base_model(input_anchor)
    net_positive = base_model(input_positive)
    net_negative = base_model(input_negative)
    if intermediate:
        return Model([input_anchor,input_positive,input_negative],
                     [net_anchor,net_positive,net_negative], name='triplet_model')
    
    d_positive = Lambda(euclidean_distance, name='d_pos')([net_anchor, net_positive])#euclidean_distance((net_anchor, net_positive))
    d_negative = Lambda(euclidean_distance, name='d_neg')([net_anchor, net_negative])#euclidean_distance((net_anchor, net_negative))
    d_stacked = Lambda(lambda vects: K.stack(vects, axis=1), name='d_stacked')([d_positive, d_negative])
    
    triplet_model = Model([input_anchor,input_positive,input_negative], d_stacked, name='triplet_model')
    return triplet_model

def euclidean_distance(vects):
    anchor, target = vects
    return K.sqrt(K.maximum(K.sum(K.square(anchor - target), axis=1, keepdims=True), K.epsilon()))
    #return K.sum(K.square(anchor - target), axis=1, keepdims=True)


def triplet_loss(y_true, y_pred):
        margin = K.constant(0.2)
        """print(y_pred[:,0,0])
        print(K.square(y_pred[:,0,0]))
        print(y_pred[:,1,0])
        print(K.square(y_pred[:,1,0]))"""
        """return K.mean(K.maximum(K.constant(0),
                                K.square(y_pred[:,0,0]) - K.square(y_pred[:,1,0]) + margin))"""
        return K.mean(K.maximum(K.constant(0), y_pred[:,0,0] - y_pred[:,1,0] + margin))
    
with tf.Session(graph=triplet_graph) as triplet_sess:
    K.set_learning_phase(1)
    # 모델 resume 필요
    triplet_model = get_triplet_model(True)
    triplet_model.compile(optimizer=Adam(lr=1e-7) ,loss='mean_squared_error')
    triplet_sess.run(tf.global_variables_initializer())

    batch = 1
    (t_lbl, t_path, p_path), (n_lbl, n_path) = random_sample(batch)

    anchor, positive, negative = images_from_paths(t_path, p_path, n_path)
    #print(anchor[0][0][0:5])
    #print(triplet_model.summary())
    #for i in range(4):
        #print(triplet_sess.run(triplet_model.layers[3].get_output_at(i),
        #                       feed_dict={'input_anchor':anchor,'input_pos':positive,'input_neg':negative}))
    #print(triplet_model.layers[len(triplet_model.layers)-1].output)
    
    a=triplet_sess.run(triplet_model.layers[len(triplet_model.layers)-1].output, feed_dict={'input_1:0':anchor})
    b=triplet_sess.run(triplet_model.layers[len(triplet_model.layers)-1].output, feed_dict={'input_1:0':positive})
    c=triplet_sess.run(triplet_model.layers[len(triplet_model.layers)-1].output, feed_dict={'input_1:0':negative})
    print(triplet_sess.run(triplet_model.layers[len(triplet_model.layers)-1].output, feed_dict={'input_1:0':anchor}))
    
    true_model = get_triplet_model()
    true_model.compile(optimizer=Adam(lr=1e-7) ,loss=triplet_loss)
    #true_model.fit(x=[anchor,positive,negative], y=np.random.randint(2, size=(1,2,1)).T)
    print(true_model.train_on_batch(x=[anchor, positive, negative],
                                            y=np.random.randint(2, size=(1,2,1)).T))

[[0.35683742 0.         0.         ... 0.         0.         0.        ]]
2.8359787


In [92]:
# 모델() 2.8
# 실계산
def np_euclidean(a,b):
    return np.sum(np.square(a-b))
def np_triplet(pos,neg):
    return np.mean(np.maximum(0,pos-neg+0.2))
from numpy import linalg as LA
from sklearn import preprocessing
#from tensorflow.contrib.losses.metric_learning import triplet_semihard_loss
d__pos = np_euclidean(a,b)
d__neg = np_euclidean(a,c)
print((d__pos,d__neg))
print(np_triplet(d__pos,d__neg))

(526.8052, 462.05658)
64.94859619140625


In [64]:
print(a.shape)


(1, 1024)


In [ ]:
        from tensorflow.contrib.losses.metric_learning import triplet_semihard_loss
        triplet_semihard_loss([1,1,2],[anchor,])

In [37]:
            """for layer in triplet_model.layers:
                print(layer)
                #print(layer.trainable)
                print(layer.output)"""
            for i in range(4):
                print(triplet_model.layers[3].get_output_at(i).eval(session=triplet_sess))#layers[3].output

RuntimeError: Attempted to use a closed Session.

In [86]:
import warnings
warnings.filterwarnings('ignore')

triplet_graph = tf.Graph()
keras_weights_path = os.path.join(os.getcwd(),'watchout/models/tripletnetwork/checkpoint')

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        # triplet_graph라는 전혀 다른 graph object를 할당한 session에서 실행
        with tf.Session(graph=triplet_graph) as triplet_sess:
            global_step = tf.get_variable('global_step', initializer=0, trainable=False)
            increment_global_step = tf.assign(global_step, global_step+1)
            
            # 모델 resume 필요
            triplet_model = get_triplet_model()
            
            triplet_sess.run(tf.global_variables_initializer())
            
            try:
                latest_weights, global_step_value = get_latest_weights_and_global_step(keras_weights_path)
                triplet_sess.run(global_step.assign(global_step_value))
            except Exception as e:
                print('No weights file!')
                print(e)
            
            print(global_step.eval())
            #c = 1/0
            
            triplet_model.compile(optimizer=Adam(lr=1e-7), loss=triplet_loss)
            #keras_tensorboard = TensorBoard(log_dir="./watchout/models/tripletnetwork/logs/{}".format(time()))
            #keras_checkpoint = ModelCheckpoint('./watchout/models/tripletnetwork/checkpoint/weights.best.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='min')
            
            train_writer = tf.summary.FileWriter('./watchout/models/tripletnetwork/logs/train', triplet_sess.graph)
            test_writer = tf.summary.FileWriter('./watchout/models/tripletnetwork/logs/test')

            batch = 1
            (t_lbl, t_path, p_path), (n_lbl, n_path) = random_sample(batch)

            anchor, positive, negative = images_from_paths(t_path, p_path, n_path)

            print('초기 anchor,positive,negative shape')
            print(anchor.shape)
            print(positive.shape)
            print(negative.shape)
            
            for idx,single_batch in enumerate(t_path):
                
                image = Image.open(single_batch)
                image_np = load_image_into_numpy_array(image)
                image_np_expanded = np.expand_dims(image_np, axis=0)
                (_image_tensor,_boxes, scores, classes, num) = sess.run(
                  [image_tensor, detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
                
                target_boxes = promising_boxes(_boxes, scores[0])
                len_target_boxes = int(target_boxes.shape[0])
                
                cropped_images = []
                for box in target_boxes.eval().tolist():
                    current_cropped_image = tf.image.crop_and_resize(image=image_np_expanded/255.0,
                                                                     boxes=[box],
                                                                     box_ind=[0],
                                                                     crop_size=[299,299])
                    cropped_images.append(current_cropped_image[0])
                    anchor= np.append(anchor,np.expand_dims(current_cropped_image[0].eval(),axis=0),axis=0)
                    positive = np.append(positive,np.expand_dims(positive[idx],axis=0),axis=0)
                    negative = np.append(negative,np.expand_dims(negative[idx],axis=0),axis=0)
            
            print('후기 anchor,positive,negative shape')
            print(anchor.shape)
            print(positive.shape)
            print(negative.shape)
            
            assert anchor.shape[0] == positive.shape[0] and positive.shape[0] == negative.shape[0]
            
            #################
            
            #anchor, positive, negative = get_train_data(batch=1)
            final_batch = anchor.shape[0]
            
            for n in range(2):
                training_loss = triplet_model.train_on_batch(x=[anchor, positive, negative],
                                            y=np.random.randint(2, size=(1,2,final_batch)).T)
                triplet_sess.run(increment_global_step)
                train_summary = tf.Summary(value=[tf.Summary.Value(tag="train_loss", 
                                             simple_value=training_loss)])
                train_writer.add_summary(train_summary,global_step.eval())
                
                print('train done')
                
                if n % 2 == 0:
                    (_t_lbl, _t_path, _p_path), (_n_lbl, _n_path) = random_sample(30)

                    _anchor, _positive, _negative = images_from_paths(_t_path, _p_path, _n_path)
                    test_loss = triplet_model.test_on_batch(x=[_anchor, _positive, _negative],
                                            y=np.random.randint(2, size=(1,2,30)).T)
                    test_summary = tf.Summary(value=[tf.Summary.Value(tag="test_loss", 
                                             simple_value=test_loss)])
                    test_writer.add_summary(test_summary,global_step.eval())
                    print('test done')

            triplet_model.save_weights(os.path.join(os.getcwd(),'watchout/models/tripletnetwork/checkpoint/weights.best.hdf5'))

No weights file!
list index out of range
0
초기 anchor,positive,negative shape
(1, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
후기 anchor,positive,negative shape
(2, 299, 299, 3)
(2, 299, 299, 3)
(2, 299, 299, 3)
train done
test done
train done


In [5]:
import warnings
warnings.filterwarnings('ignore')

"""triplet_graph = tf.Graph()
keras_weights_path = os.path.join(os.getcwd(),'watchout/models/tripletnetwork/checkpoint')

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        
# triplet_graph라는 전혀 다른 graph object를 할당한 session에서 실행
def get_latest_weights_and_global_step(_path):
    files = os.listdir(path=_path)
    files.reverse()
    return files[0],int(files[0].split('-')[1].split('.')[0])

with tf.Session(graph=triplet_graph) as triplet_sess:
    global_step = tf.get_variable('global_step', initializer=0, trainable=False)
    increment_global_step = tf.assign(global_step, global_step+1)

    # 모델 resume 필요
    #triplet_model = get_triplet_model()
    triplet_sess.run(tf.global_variables_initializer())
    try:
        latest_weights, global_step_value = get_latest_weights_and_global_step(keras_weights_path)
        #triplet_model.load_weights(latest_weights)
        triplet_sess.run(global_step.assign(global_step_value))
        print(latest_weights)
        print(global_step_value)
    except Exception as e:
        print('except')
        print(e)
    print(global_step.eval())"""
def image_from_path(target, positive, negative):
    tmp_a = Image.open(target)
    tmp_p = Image.open(positive)
    tmp_n = Image.open(negative)
    
    # arr = array(img, dtype="float32")
    
    anchor = tmp_a.copy()
    anchor = anchor.resize((299, 299), Image.ANTIALIAS)
    anchor = np.array(anchor, dtype="float32")
    pos = tmp_p.copy()
    pos = pos.resize((299,299), Image.ANTIALIAS)
    pos = np.array(pos, dtype="float32")
    neg = tmp_n.copy()
    neg = neg.resize((299,299), Image.ANTIALIAS)
    neg = np.array(neg, dtype="float32")
    
    tmp_a.close()
    tmp_p.close()
    tmp_n.close()
    
    return anchor/255.0, pos/255.0, neg/255.0

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        a,p,n=get_train_data(1,sess)
        print(a.shape)
        print(p.shape)
        print(n.shape)

NameError: name 'get_train_data' is not defined

In [102]:
        p[0]

array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       ...,

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)

In [89]:
def get_latest_weights_and_global_step(_path):
    files = os.listdir(path=_path)
    print('weights list:'+str(files))
    files.reverse()
    print(files)
    return files[0],int(files[0].split('-')[1].split('.')[0])
print(get_latest_weights_and_global_step(keras_weights_path))

weights list:['.DS_Store', 'weights-0.hdf5', 'weights-99.hdf5', 'weights.best.hdf5']
['weights.best.hdf5', 'weights-99.hdf5', 'weights-0.hdf5', '.DS_Store']


IndexError: list index out of range

In [48]:
            triplet_model.save_weights(os.path.join(os.getcwd(),'watchout/models/tripletnetwork/checkpoint/weights.best.hdf5'))

ValueError: Fetch argument <tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 3, 32) dtype=float32_ref> cannot be interpreted as a Tensor. (Tensor Tensor("conv2d_1/kernel:0", shape=(3, 3, 3, 32), dtype=float32_ref) is not an element of this graph.)